# Evaluating error terms



In [1]:
cd ..

/ihome/ltang/ziz60/nmf_causal_eval


In [2]:
import numpy as np
import copy
from helper_functions import reorder_V_val, generate_data_dlearning,reorder_U, reorder_V, reorder_beta, reorder_beta_ls, pos, neg, NNDSVD, vec2col, error_plot, find_matched_topic_index


import time
from numpy.random import rand
from numpy import exp, log, mat
from numpy.linalg import norm
from scipy.io import loadmat
from scipy.linalg import svd 
from array import array
import pandas as pd


import math
from itertools import permutations #used in find_matched_topic_index


from sklearn.decomposition import NMF # Used in two-stage NMF
#from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split #for cross validation


## my functions
#from helper_functions import pos, neg, NNDSVD, vec2col, error_plot

#from sklearn.metrics import adjusted_rand_score
#from helper_functions_eval import reorder_U_val, reorder_U, reorder_V, reorder_beta,vec2col, generate_data_dlearning,pos, neg, NNDSVD, vec2col, error_plot,find_optimal_r
#from evaluation import eval_ITR, find_matched_topic_index, eval_overall_congruence,eval_topic_congruence, eval_ari, eval_topic_selection,Dlearn_SNMF_eval 
from evaluation import eval_ITR
from PGD_Dlearning_v3 import objective_function, objective_U, objective_V, objective_b2, subprob_U, subprob_V, subprob_b2, gradient_descent
from PGD_Dlearning_test import gradient_descent_test

from sklearn.linear_model import LogisticRegression


from itertools import permutations

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm

import time
from datetime import datetime

# Define a function to assign max_corr_topic for each group
def assign_max_corr_topic(group):
    # Sort the group by 'value' column in descending order
    group_sorted = group.sort_values(by='value', ascending=False)
    
    # Assign max_corr_topic for each row
    group_sorted['max_corr_topic'] = range(1, R+1)
        
    return group_sorted

import warnings


def count_matches(perm, df, choices):
    column_name = "m"
    for choice in perm:
        column_name += str(choice)
        conditions = [df["topic_hat"] == int(choice)]
        df[column_name] = np.select(conditions, choices, default=np.nan)
    columns_to_check = df.columns[2:]

    counts = []
    for column_name in columns_to_check:
        count = len(df[df['topic_true'] == df[column_name]])
        counts.append(count)
    return max(counts), perm
        
def largest_absolute_indices(arr, k):
    # Get the indices that would sort the absolute values of the array in descending order
    sorted_indices = np.argsort(np.abs(arr))[::-1]

    # Select the first r indices
    largest_indices = sorted_indices[:k]
    # Convert indices to a Python list
    largest_indices_list = largest_indices.tolist()
    return largest_indices_list


def plot(word_key, convergence_step, V_ls, ax=None):
    word_ls = []
    for i in range(convergence_step):
        word = V_ls[i][:, word_key-1]  # Assuming word_key is 1-indexed
        word = word.reshape(-1, 1)
        word_ls.append(word)

    word_df = np.hstack(word_ls)  
    word_df = pd.DataFrame(word_df.T, columns=['T1', 'T2', 'T3', 'T4', 'T5'])
    word_df['iteration'] = range(convergence_step)
    word_df_long = word_df.melt(id_vars='iteration', var_name='topic', value_name='V')

    # Create the line plot
    if ax is None:
        fig, ax = plt.subplots()
    sns.lineplot(data=word_df_long, x='iteration', y='V', hue='topic', marker=None, ax=ax, legend=None)

    # Add title and labels
    ax.set_title(f'Word {word_key+1}')
    ax.set_xlabel('Iteration')
    ax.set_ylim(-0.1, 1.5)


from plot import plot_V, plot_tuning,plot_tmt_propensity

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from joblib import dump, load

In [3]:
def run_cv_one_sim(seed_train, seed_validation, seed_test, nmf_random_seed, r, tol, maxiter, maxiter_sub, print_message_snmf,alpha_values, lambda_values, v_signal_val, eta, epsilon, step_size_a, step_size_b, step_size_sigma, step_size ): # Scenerio A
    export_name = f'/ix1/ltang/ziz60/nmf_causal_eval/21_simple_scenario_interaction/output18/scenario2_r{r}_sim_{seed_train}_{seed_validation}_{seed_test}.joblib'

    print("Save output called:", export_name)
    
    main_topic_signal = 10
    topic_weights = [2,2, 1, 3, 4]  # Adjust weights as desired

    # RCT
    gamma1 = gamma2 = 0 

    psi1= 10
    psi2= 5
    beta0 = 0
    beta1 = 0
    beta2 = 0
    beta3 = -30
    beta4 = 0
    beta5 = 0
    rho1 = 0 #Interaction effect between words in topic 3 and topic 4: X2_41*X2_42*X2_61* X2_62 
    rho2 = 0 #Interaction effect between words in topic 1 and 2
    rho3 = 0 #TBD

    n, m1, m2, R = 2000, 10, 100, 5

    eta= eta #controls noise level 
    epsilon= epsilon #controls noise level 

    v_signal_val = v_signal_val  #words 1, 2, 21, 22's values in V matrix.
    correlation_type = 6
    
    ################################################################## Train
    data_train, Vtrue_train, NA, rmse_min_train= generate_data_dlearning(seed=seed_train, n = n, r=R, m1=m1, m2 = m2, zeta1= 1, zeta2=2,\
                                    main_topic_signal=main_topic_signal,topic_weights \
                                    = topic_weights,eta =eta, epsilon=epsilon, gamma1=gamma1, gamma2=gamma2, \
                                    beta0=beta0, psi1 = psi1, psi2=psi2,beta1=beta1, beta2=beta2,  beta3=beta3, beta4 = beta4, beta5 = beta5, rho1 =rho1, rho2=rho2, rho3= rho3, v_signal_val=v_signal_val,scenario="A", sparse = True, correlation_type=correlation_type)
    #print("Train:")
    #print(pd.crosstab(index = data_train['d_x'], columns = 'count')) # Treatment Assignment, controlled by betas
    #plt.hist(data_train['delta_x'], bins = 20)

    # Initialize the list to store the generated data
    validation_datasets = []
    Vtrue_validations = []
    rmse_min_vals = []

    # Loop to generate datasets with incrementing seeds
    for k in range(10):
        seed = seed_validation + k  # Increment seed by 1 for each dataset
        data_validation, Vtrue_validation, _, rmse_min_val = generate_data_dlearning(
            seed=seed, n=n, r=R, m1=m1, m2=m2, zeta1=1, zeta2=2,
            main_topic_signal=main_topic_signal, topic_weights=topic_weights, eta=eta, epsilon=epsilon,
            gamma1=gamma1, gamma2=gamma2, beta0=beta0, psi1=psi1, psi2=psi2,
            beta1=beta1, beta2=beta2, beta3=beta3, beta4=beta4, beta5=beta5,
            rho1=rho1, rho2=rho2, rho3=rho3, v_signal_val=v_signal_val, scenario="A", sparse = True, correlation_type=correlation_type
        )

        # Store the generated data in the lists
        validation_datasets.append(data_validation)
        Vtrue_validations.append(Vtrue_validation)
        rmse_min_vals.append(rmse_min_val)
    
    data_test, Vtrue_test, NA, rmse_min_test = generate_data_dlearning(seed=seed_test, n = n, r=R, m1=m1, m2 = m2, zeta1= 1, zeta2=2,\
                                main_topic_signal=main_topic_signal,topic_weights \
                                = topic_weights,eta =eta, epsilon=epsilon, gamma1=gamma1, gamma2=gamma2, \
                                beta0=beta0, psi1 = psi1, psi2=psi2,beta1=beta1, beta2=beta2,  beta3=beta3, beta4 = beta4, beta5 = beta5, rho1 =rho1, rho2=rho2, rho3= rho3, v_signal_val=v_signal_val,scenario="A", sparse = True, correlation_type=correlation_type)

    print(pd.crosstab(index = data_train['d_x'], columns = 'count')) # Treatment Assignment, controlled by betas
    print( "minimum RMSE train =" , rmse_min_train)
    print( "minimum RMSE val acroos 10 validations=" , rmse_min_vals)
    print( "minimum RMSE test =" , rmse_min_test)
    ####################################################### Training
    delta_x = data_train["delta_x"]
    delta_x = vec2col(delta_x)
    d_x = data_train["d_x"]

    Y = data_train[['y']].values #array
    A = data_train[['A']].values #array
    X = data_train.iloc[:, 0:m1+m2] 
    X1 =  data_train.iloc[:, 0:m1] #EHR data
    X2 =  data_train.iloc[:, m1:m1+m2] #document word 
    U = data_train.iloc[:, (m1+m2+1):(m1+m2 +R+1)] #no intercept

    # format dataframe to np array
    X = X.to_numpy()
    X1 = X1.to_numpy()
    X2 = X2.to_numpy()
    U = U.to_numpy()
    U_true = U.copy()


    # INIT U, V from X2
    model = NMF(n_components= r, init='random', random_state= nmf_random_seed)
    Uinit = model.fit_transform(X2) #n times r
    Vinit = model.components_

    ##### Obtain Y_tilde
    # estimate treatment propensity
    X = np.concatenate((X1, Uinit), axis=1)
    if gamma1 == gamma2 == 0: 
    # RCT, random treatment assignment
        data_train["pi_hat"] = 0.5
        print("RCT, P(A=1) = 0.5.")
    else:
        print("Observational study, use logistic regression to estimate P(A).")
        # estimate treatment propensity
        #estimate p_hat using logistic regression, regression data on X (EHR and initial U)
        model = LogisticRegression(solver='liblinear', random_state=0)
        model.fit(X, A)
        #model.fit(X1, A) # only using EHR to predict treatment assignment

        pi_hat = model.predict_proba(X)[:, 1] #p_hat = estiamted probability A=1
        #pi_hat = model.predict_proba(X1)[:, 1] #p_hat = estiamted probability A=1
        pi_hat = vec2col(pi_hat)
        data_train["pi_hat"] = pi_hat
        # Create scatter plot
        plot_tmt_propensity(data_train)

    W_hat = np.where(data_train['A'] == 1, 1/data_train['pi_hat'], 1/(1 - data_train['pi_hat']))
    data_train['W_hat'] = W_hat
    W_hat = vec2col(W_hat)
    W = np.eye(len(W_hat)) # make W a diagonal matrix
    np.fill_diagonal(W, W_hat)

    Y_tilde = 2*data_train['y']*data_train['A'] * np.sqrt(data_train["W_hat"]) # transform Y
    Y_tilde = vec2col(Y_tilde)


    ############################################################
    num_alphas = len(alpha_values)


    ##################################### INIT beta
    beta_init = np.zeros((1+m1+r, 1))
    beta_init[0] = 0

    beta_init = vec2col(beta_init)
    ########################################
    output = [[None] * len(lambda_values)  for _ in range(len(alpha_values))]
    #output[0][0] --> outout of the first lambda for the first alpha

    results = []

    ##################################################################################### Train
    for i, alpha1 in enumerate(alpha_values):
        print("***********Tuning: alpha1=", alpha1, "***********")
        for j, lambda_i in enumerate(lambda_values):
            print("**Tuning: lambda =", lambda_i, "**")

            convergence_step, rmse, x2_minus_uv_norm, v_minus_vhat_norm,specificity, sensitivity, accuracy, Y_tilde_hat, delta_x_hat, U_out, V_out, b2_out, projnorm_ls, objective_new_ls, b2_ls, gradU_matrix, objU_matrix, gradV_matrix, objV_matrix, gradb2_matrix, objb2_matrix, V_ls, U_ls_train \
                = gradient_descent(X2, X, W, A, Y, Y_tilde, Uinit, Vinit, Vtrue_train, r, beta_init, alpha1, lambda_i, lambda_i, lambda_i, d_x, tol, maxiter,maxiter_sub, correction = False, print_message_snmf =True, print_message=False, a=step_size_a, b=step_size_b, sigma = step_size_sigma, step_size=step_size, eval = True)

            if convergence_step > maxiter:
                convergence_step = f"reach max iteration: {maxiter}"
            # Append alpha value and its corresponding evaluation_y to the results list
            results.append({'alpha1': alpha1, 
                            'lambda': lambda_i,
                            'convergence_step': convergence_step,
                            'rmse': rmse,
                            'x2_minus_uv_norm': x2_minus_uv_norm,
                            'v_minus_vhat_norm': v_minus_vhat_norm,
                            'accuracy_dx': accuracy,
                           'specificity': specificity ,
                            'sensitivity': sensitivity,
                            'classification_error': 1-accuracy})
            output[i][j] = Y_tilde_hat, delta_x_hat, U_out, V_out, b2_out, projnorm_ls, objective_new_ls, b2_ls, gradU_matrix, objU_matrix, gradV_matrix, objV_matrix, gradb2_matrix, objb2_matrix, V_ls, U_ls_train

    train_results = pd.DataFrame(results)

    ############################################################
    ####################    Validation      ####################
    ############################################################
    maxiter_val = 1500  # max number of iterations for validation

    def perform_validation(data_validation, V_out, Vtrue_train, Vtrue_validation, r, b2_out, alpha1, lambda_i, maxiter_val):
        delta_x = vec2col(data_validation["delta_x"])
        d_x = data_validation["d_x"]
        Y = data_validation[['y']].values
        A = data_validation[['A']].values
        X1 = data_validation.iloc[:, 0:m1].to_numpy()
        X2 = data_validation.iloc[:, m1:m1+m2].to_numpy()

        # Initialize U, prep for optimization
        model = NMF(n_components=r, init='random', random_state=1)
        Uinit = model.fit_transform(X2)

        # Obtain W_hat from validation
        X = np.concatenate((X1, Uinit), axis=1)
        if gamma1 == gamma2 == 0:
            data_validation["pi_hat"] = 0.5
        else:
            model = LogisticRegression(solver='liblinear', random_state=0)
            model.fit(X, A)
            pi_hat = model.predict_proba(X)[:, 1]
            data_validation["pi_hat"] = vec2col(pi_hat)

        W_hat = np.where(data_validation['A'] == 1, 1/data_validation['pi_hat'], 1/(1 - data_validation['pi_hat']))
        data_validation['W_hat'] = W_hat
        W_hat = vec2col(W_hat)
        W = np.eye(len(W_hat))
        np.fill_diagonal(W, W_hat)

        Y_tilde = 2 * data_validation['y'] * data_validation['A'] * np.sqrt(data_validation["W_hat"])
        Y_tilde = vec2col(Y_tilde)

        U_out_validation, convergence_step = gradient_descent_test(
            X2, X, W, Y, Y_tilde, Uinit, V_out, Vtrue_train, r, b2_out, alpha1, lambda_i, tol,
            maxiter_val, step_size_a, step_size_b, step_size_sigma, False
        )
        
        # X2- UV errors
        x2_minus_uv =  X2 - np.dot(U_out_validation, V_out)
        x2_minus_uv_norm = np.linalg.norm(x2_minus_uv, 'fro')/(r*m2)

        # V error
        V_out_R = reorder_V(Vtrue_train, V_out, m2, False)
        v_minus_vhat = Vtrue_validation - V_out_R
        v_minus_vhat_norm = np.linalg.norm(v_minus_vhat, 'fro')/(r*m2)
       
        
        U_out_validation_r = reorder_U(Vtrue_train, V_out, U_out_validation, m2, True)
        b2_out_r = reorder_beta(Vtrue_train, V_out, b2_out, m1, m2, True)

        delta_x_hat_r = b2_out_r[0] + X1.dot(b2_out_r[1:m1+1]) + U_out_validation_r.dot(b2_out_r[m1+1:])
        d_x_hat_r = np.where(delta_x_hat_r > 0, 1, -1).squeeze()

        _, _, _, _, specificity_r, sensitivity_r, accuracy_r = eval_ITR(d_x, d_x_hat_r)
        
        diff = 2 * Y * A - delta_x_hat_r
        rmse_r = np.sqrt(np.mean(diff ** 2))

        return convergence_step, rmse_r, x2_minus_uv_norm, v_minus_vhat_norm, accuracy_r, specificity_r, sensitivity_r, U_out_validation

    all_validation_results = []

    # Assuming alpha_values, lambda_values, and validation_datasets are already defined
    n_alpha = len(alpha_values)        # Number of alpha values
    n_lambda = len(lambda_values)      # Number of lambda values
    n_validation = len(validation_datasets)  # Number of validation datasets (e.g., 10)

    # Initialize U_validation_ls as a nested list
    U_validation_ls = [[[None for _ in range(n_validation)] for _ in range(n_lambda)] for _ in range(n_alpha)]

    for i, alpha1 in enumerate(alpha_values):
        for j, lambda_i in enumerate(lambda_values):
            print(f"Validation: alpha1={alpha1}, lambda={lambda_i}")
            Y_tilde_hat, delta_x_hat, U_out, V_out, b2_out, *_ = output[i][j]

            for k, data_validation in enumerate(validation_datasets): #validation_datasets is a list of 10 validation data
                # Perform validation
                Vtrue_validation = Vtrue_validations[k]
                convergence_step, rmse_r, x2_minus_uv_norm, v_minus_vhat_norm, accuracy_r, specificity_r, sensitivity_r, U_out_validation = perform_validation(
                    data_validation, V_out, Vtrue_train, Vtrue_validation, r, b2_out, alpha1, lambda_i, maxiter_val
                )
                
                U_validation_ls[i][j][k] = U_out_validation
                
                # Store results in a dictionary and append to the list
                all_validation_results.append({
                    'validation_set': k+1,
                    'alpha1': alpha1, 
                    'lambda': lambda_i,
                    'convergence_step': convergence_step,
                    'rmse': rmse_r,
                    'x2_minus_uv_norm': x2_minus_uv_norm,
                    'v_minus_vhat_norm': v_minus_vhat_norm,
                    'accuracy_dx': accuracy_r, 'specificity': specificity_r, 'sensitivity': sensitivity_r,
                    'classification_error': 1 - accuracy_r
                })

    validation_results = pd.DataFrame(all_validation_results)

    #print(validation_results)
    
    ############################################################ 
    ############   Optimal Hyperparameters         #############
    ############################################################ 

#     validation_avg = validation_results.groupby(['alpha1', 'lambda']).agg(
#         rmse_mean=('rmse', 'mean'),
#         rmse_sd=('rmse', 'std'),
#         classification_error_mean=('classification_error', 'mean'),
#         x2_minus_uv_norm_mean = ('x2_minus_uv_norm', 'mean'),
#         v_minus_vhat_norm_mean = ('v_minus_vhat_norm', 'mean')
#     ).reset_index()
#     validation_avg['rmse_lb'] = validation_avg['rmse_mean'] - validation_avg['rmse_sd']
#     validation_avg['rmse_ub'] = validation_avg['rmse_mean'] + validation_avg['rmse_sd']

#     ##########################################
#     min_rmse_mean = validation_avg['rmse_mean'].min()
#     min_rmse_row = validation_avg[validation_avg['rmse_mean'] == min_rmse_mean]

#     lb = min_rmse_row['rmse_lb'].values[0]
#     ub = min_rmse_row['rmse_ub'].values[0]
#     #############################################
   
#     validation_avg['within_1sd_TF'] = (validation_avg['rmse_mean'] > lb) & (validation_avg['rmse_mean'] < ub)
#     print(validation_avg)
        
#     validation_avg_true = validation_avg[validation_avg['within_1sd_TF'] == True]
#     ordered_validation_avg = validation_avg_true.sort_values(by=['alpha1', 'lambda'], ascending=[True, True])
#     print(ordered_validation_avg)

#     first_row = ordered_validation_avg.iloc[0]
    
#     # optimal hyperparameters (alpha and lambda) values
#     alpha_star = first_row['alpha1']
#     lambda_star =first_row['lambda']

#     # the index of the optimal hyperparameters
#     alpha_star_index = alpha_values.index(alpha_star)
#     lambda_star_index = lambda_values.index(lambda_star)

#     print("Optimal alpha = ", alpha_star, "(index", alpha_star_index,")", "\nOptimal lambda=", lambda_star, "(index", lambda_star_index,")")

#     optimal_hyperparam_report = []
#     optimal_hyperparam_report.append({'optimal_alpha': alpha_star, 
#                                       'optimal_lambda': lambda_star,
#                                        'optimal_alpha_index': alpha_star_index,
#                                        'optimal_lambda_index': lambda_star_index})
#     optimal_hyperparam_report= pd.DataFrame(optimal_hyperparam_report)

     ############################################################ 
    ####################    Testing         ####################
    ############################################################ 
    test_results = []

    U_test_ls = [[None] * len(lambda_values)  for _ in range(len(alpha_values))]

    # Loop over all alpha_star_index and lambda_star_index combinations
    for i, alpha_i in enumerate(alpha_values):
        for j, lambda_j in enumerate(lambda_values):

            # Extract the optimal output for the current alpha and lambda
            output_optimal = output[i][j]
            V_snmf = output_optimal[3]
            beta_snmf = output_optimal[4]

            # Initialize for the current alpha and lambda
            maxiter_val_test = 1000
            delta_x = data_test["delta_x"]
            delta_x = vec2col(delta_x)
            d_x = data_test["d_x"]

            Y = data_test[['y']].values  # Convert to numpy array
            A = data_test[['A']].values  # Convert to numpy array
            X1 = data_test.iloc[:, 0:m1].to_numpy()  # EHR data
            X2 = data_test.iloc[:, m1:m1+m2].to_numpy()  # Document word data

            n = data_test.shape[0] 

            # Initialize U
            model = NMF(n_components=r, init='nndsvd', random_state=1)
            Uinit = model.fit_transform(X2)

            # Estimate W_hat
            X = np.concatenate((X1, Uinit), axis=1)
            if gamma1 == gamma2 == 0:
                data_test["pi_hat"] = 0.5  # RCT, random treatment assignment
            else:
                # Estimate treatment propensity using logistic regression
                model = LogisticRegression(solver='liblinear', random_state=0)
                model.fit(X, A)
                pi_hat = model.predict_proba(X)[:, 1]  # Estimated probability A=1
                pi_hat = vec2col(pi_hat)
                data_test["pi_hat"] = pi_hat

            W_hat = np.where(data_test['A'] == 1, 1/data_test['pi_hat'], 1/(1 - data_test['pi_hat']))
            data_test['W_hat'] = W_hat
            W_hat = vec2col(W_hat)
           
            W = np.eye(len(W_hat))  # Make W a diagonal matrix
            np.fill_diagonal(W, W_hat)

            # Compute Y_tilde
            Y_tilde = 2 * data_test['y'] * data_test['A'] * np.sqrt(data_test["W_hat"])
            Y_tilde = vec2col(Y_tilde)

            # Optimize U given X2 and V_snmf
            U_out_test, convergence_step = gradient_descent_test(
                X2, X, W, Y, Y_tilde, Uinit, V_snmf, Vtrue_train, r, beta_snmf, 
                0, lambda_j, tol, maxiter_val_test, step_size_a, step_size_b, 
                step_size_sigma, False
            )

            U_test_ls[i][j] = U_out_test
            
            # Reorder U and other variables
            U_out_test_r = reorder_U(Vtrue_train, V_snmf, U_out_test, m2, True)
            U_out_test_R = reorder_U(Vtrue_train, V_snmf, U_out_test, m2, False)

            beta_snmf_r = reorder_beta(Vtrue_train, V_snmf, beta_snmf, m1, m2, True)
            beta_snmf_R = reorder_beta(Vtrue_train, V_snmf, beta_snmf, m1, m2, False)

            V_out_r = reorder_V(Vtrue_train, V_snmf, m2, True)
            V_out_R = reorder_V(Vtrue_train, V_snmf, m2, False)

            # Evaluation Criteria
            # Compute delta_x_hat
            delta_x_hat = beta_snmf_r[0] + X1.dot(beta_snmf_r[1:m1+1]) + U_out_test_r.dot(beta_snmf_r[m1+1:])
            d_x_hat = np.where(delta_x_hat > 0, 1, -1).squeeze()

            # Evaluate ITR
            NA, NA, NA, NA, specificity, sensitivity, accuracy = eval_ITR(d_x, d_x_hat)

            # RMSE
            diff = 2 * Y * A - delta_x_hat
            rmse = np.sqrt(np.mean(diff**2))

            # 3. V Error
            if r < R:
                v_minus_vhat = Vtrue_train - V_out_R
            else:
                v_minus_vhat = Vtrue_train - V_out_r
            v_minus_vhat_norm = np.linalg.norm(v_minus_vhat, 'fro')/(r*m2)


            # X2-UV norm
            x2_minus_uv =  X2 - np.dot(U_out_test_r, V_out_r)
            x2_minus_uv_norm = np.linalg.norm(x2_minus_uv, 'fro')/(n*m2)

            # Store the test results for the current alpha and lambda combination
            test_results_i = {
                'alpha':alpha_i,  # Fetch the actual alpha value
                'lambda': lambda_j,  # Fetch the actual lambda value
                'convergence_step': convergence_step,
                'rmse': rmse,
                'accuracy_dx': accuracy,
                'specificity': specificity,
                'sensitivity': sensitivity,
                'classification_error': 1 - accuracy,
                'v_minus_vhat_norm':v_minus_vhat_norm,
                'x2_minus_uv_norm': x2_minus_uv_norm
            }

            # Append the result to the main results list
            test_results.append(test_results_i)

    # Convert the final results to a DataFrame
    test_results = pd.DataFrame(test_results)


    dump([output, train_results, validation_results, test_results, U_validation_ls, U_test_ls], export_name)
    
    return(output, train_results, validation_results, test_results, U_validation_ls, U_test_ls)


In [4]:
alpha_values = [1e-7, 1e-3, 0.1, 1] 

lambda_values  = [1e-10, 1e-5]

# Parallel Computing

- maxiter = 1000
- maxiter_sub = 300

In [5]:
import concurrent.futures

# Define your simulation function
def run_simulation(sim):
    print(f"-------------------------------Running simulation {sim + 1}-------------------------------------------------")
    output, train_results, validation_results, test_results, U_validation_ls, U_test_ls = \
        run_cv_one_sim(seed_train=30+ sim, seed_validation= 130 + sim, seed_test= 1030 + sim, \
                       nmf_random_seed=0, r=5, tol=1e-10, maxiter=1000, maxiter_sub=300, print_message_snmf=True, \
                       alpha_values=alpha_values, lambda_values=lambda_values,\
                       v_signal_val=5, eta=1, epsilon=0.2, \
                       step_size_a=1, step_size_b=0.1, step_size_sigma=0.01, step_size=1)

    return output, train_results, validation_results, test_results,U_validation_ls, U_test_ls


In [ ]:

start_time = time.time()
print("Current time:", datetime.now())

# Initialize lists to store the results of each simulation
outputs = []
train_results_list = []
validation_results_list = []
test_results_list = []
U_validation_ls_list = []
U_test_ls_list = []


# Run the simulation 100 times
num_sim = 30

with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit tasks to the executor
    future_to_sim = {executor.submit(run_simulation, sim): sim for sim in range(num_sim)}

    # Collect results as they complete
    for future in concurrent.futures.as_completed(future_to_sim):
        sim = future_to_sim[future]
        try:
            results = future.result()
            # Store the results
            outputs.append(results[0])
            train_results_list.append(results[1])
            validation_results_list.append(results[2])
            test_results_list.append(results[3])
            U_validation_ls_list.append(results[4])
            U_test_ls_list.append(results[5])

        except Exception as exc:
            print(f'Simulation {sim} generated an exception: {exc}')

print("All simulations completed.")

end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print("Runtime:", runtime/3600, "hours")

print("Current time:", datetime.now())

Current time: 2024-11-01 16:15:49.473399
-------------------------------Running simulation 1-------------------------------------------------
Save output called: /ix1/ltang/ziz60/nmf_causal_eval/21_simple_scenario_interaction/output18/scenario2_r5_sim_30_130_1030.joblib
-------------------------------Running simulation 2-------------------------------------------------
Save output called: /ix1/ltang/ziz60/nmf_causal_eval/21_simple_scenario_interaction/output18/scenario2_r5_sim_31_131_1031.joblib
-------------------------------Running simulation 3-------------------------------------------------
Save output called: /ix1/ltang/ziz60/nmf_causal_eval/21_simple_scenario_interaction/output18/scenario2_r5_sim_32_132_1032.joblib
-------------------------------Running simulation 4-------------------------------------------------
Save output called: /ix1/ltang/ziz60/nmf_causal_eval/21_simple_scenario_interaction/output18/scenario2_r5_sim_33_133_1033.joblib
-------------------------------Running 

## END

---------------------------------

In [8]:
pd.DataFrame(train_results_list[0])

,alpha1,lambda,convergence_step,rmse,x2_minus_uv_norm,v_minus_vhat_norm,accuracy_dx,specificity,sensitivity,classification_error
0,1.000000e-10,1.000000e-10,reach max iteration: 1,22.368073,0.000269,0.007896,0.893,0.939769,0.821066,0.107
1,1.000000e-10,1.000000e-05,reach max iteration: 1,26.075364,0.000269,0.007897,0.893,0.939769,0.821066,0.107
2,1.000000e-07,1.000000e-10,reach max iteration: 1,22.368073,0.000269,0.007896,0.893,0.939769,0.821066,0.107
3,1.000000e-07,1.000000e-05,reach max iteration: 1,22.368084,0.000269,0.007897,0.893,0.939769,0.821066,0.107
4,1.000000e-03,1.000000e-10,reach max iteration: 1,22.368073,0.000269,0.007896,0.893,0.939769,0.821066,0.107
5,1.000000e-03,1.000000e-05,reach max iteration: 1,22.368084,0.000269,0.007897,0.893,0.939769,0.821066,0.107
6,1.000000e-01,1.000000e-10,reach max iteration: 1,22.368073,0.000269,0.007896,0.893,0.939769,0.821066,0.107
7,1.000000e-01,1.000000e-05,reach max iteration: 1,22.368084,0.000269,0.007897,0.893,0.939769,0.821066,0.107


In [10]:
pd.DataFrame(validation_results_list[0])

,validation_set,alpha1,lambda,convergence_step,rmse,x2_minus_uv_norm,v_minus_vhat_norm,accuracy_dx,specificity,sensitivity,classification_error
0,1,1.000000e-10,1.000000e-10,0,22.675962,0.116137,0.007003,0.8920,0.939570,0.819444,0.1080
1,2,1.000000e-10,1.000000e-10,0,22.450054,0.113875,0.007002,0.8940,0.935108,0.832080,0.1060
2,3,1.000000e-10,1.000000e-10,0,22.865203,0.116203,0.007032,0.9040,0.938301,0.847074,0.0960
3,4,1.000000e-10,1.000000e-10,0,22.579287,0.115515,0.007003,0.9070,0.940989,0.849866,0.0930
4,5,1.000000e-10,1.000000e-10,0,22.558861,0.115552,0.007009,0.8995,0.939268,0.839599,0.1005
...,...,...,...,...,...,...,...,...,...,...,...
75,6,1.000000e-01,1.000000e-05,0,22.565529,0.115366,0.007024,0.8970,0.937235,0.839220,0.1030
76,7,1.000000e-01,1.000000e-05,0,23.067643,0.112984,0.007017,0.9025,0.936521,0.850064,0.0975
77,8,1.000000e-01,1.000000e-05,0,22.679750,0.115017,0.007005,0.8985,0.937040,0.837838,0.1015
78,9,1.000000e-01,1.000000e-05,0,22.863017,0.115775,0.007035,0.8950,0.942691,0.822864,0.1050


In [12]:
pd.DataFrame(test_results_list[0])

,alpha,lambda,convergence_step,rmse,accuracy_dx,specificity,sensitivity,classification_error,v_minus_vhat_norm,x2_minus_uv_norm
0,1.000000e-10,1.000000e-10,0,23.141088,0.914,0.947883,0.860104,0.086,0.006998,0.000303
1,1.000000e-10,1.000000e-05,0,26.858317,0.914,0.947883,0.860104,0.086,0.006998,0.000303
2,1.000000e-07,1.000000e-10,0,23.141088,0.914,0.947883,0.860104,0.086,0.006998,0.000303
3,1.000000e-07,1.000000e-05,0,23.141090,0.914,0.947883,0.860104,0.086,0.006998,0.000303
4,1.000000e-03,1.000000e-10,0,23.141088,0.914,0.947883,0.860104,0.086,0.006998,0.000303
5,1.000000e-03,1.000000e-05,0,23.141090,0.914,0.947883,0.860104,0.086,0.006998,0.000303
6,1.000000e-01,1.000000e-10,0,23.141088,0.914,0.947883,0.860104,0.086,0.006998,0.000303
7,1.000000e-01,1.000000e-05,0,23.141090,0.914,0.947883,0.860104,0.086,0.006998,0.000303


# Evaluation

In [ ]:
cd ..

In [ ]:
from helper_functions import find_matched_topic_index, reorder_U_val, reorder_V_val, reorder_beta_val, generate_cor_df, custom_pearsonr, reorder_U, reorder_V, reorder_beta,vec2col, generate_data_dlearning,pos, neg, NNDSVD, vec2col, error_plot,find_optimal_r

from evaluation import eval_ITR

from joblib import dump, load
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


import numpy as np

In [ ]:

export_name = f'/ihome/ltang/ziz60/nmf_causal_eval/21_simple_scenario_interaction/output/scenarioA_r5_sim_0_100_1000.joblib'

print(export_name)

In [ ]:
loaded_objects = load(export_name)
optimal_hyperparam_report, output, train_results, validation_results, test_results = loaded_objects


In [ ]:
train_results

In [ ]:
alpha_values = train_results['alpha1']
alpha_values

## 1. Training Results

In [ ]:
train_results['alpha1'] = train_results['alpha1'].astype(str)
#train_results['alpha1'] = pd.Categorical(train_results['alpha1'].astype(str), categories=alpha_values, ordered=True)


train_results['alpha1'] 

In [ ]:
# Create the first plot with rmse
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot rmse on the left y-axis
sns.pointplot(x='alpha1', y='rmse', data=train_results, marker='o', linestyles='-', ax=ax1, color='b')
ax1.set_xlabel('alpha1')
ax1.set_ylabel('rmse', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create a second y-axis for x2_minus_uv_norm
ax2 = ax1.twinx()
sns.pointplot(x='alpha1', y='x2_minus_uv_norm', data=train_results, marker='o', linestyles='-', ax=ax2, color='r')
ax2.set_ylabel('x2_minus_uv_norm', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add titles
plt.title('Training - alpha vs RMSE and x2_minus_uv_norm')

# Show plot
plt.show()

In [ ]:
# Create the first plot with rmse
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot rmse on the left y-axis
sns.pointplot(x='alpha1', y='rmse', data=train_results, marker='o', linestyles='-', ax=ax1, color='b')
ax1.set_xlabel('alpha1')
ax1.set_ylabel('rmse', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create a second y-axis for x2_minus_uv_norm
ax2 = ax1.twinx()
sns.pointplot(x='alpha1', y='v_minus_vhat_norm', data=train_results, marker='o', linestyles='-', ax=ax2, color='orange')
ax2.set_ylabel('v_minus_vhat_norm', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

# Add titles
plt.title('Training - alpha  vs RMSE and v_minus_vhat_norm')

# Show plot
plt.show()

## 2. Validation results

In [ ]:
validation_results

In [ ]:
############################################################### PLOT 1
plt.figure(figsize=(12, 8))  # Adjust the width and height as needed

# Create the boxplot with dots
sns.boxplot(x='alpha1', y='rmse', data=validation_results, palette="Set2")
sns.stripplot(x='alpha1', y='rmse', data=validation_results, color='black', jitter=True, alpha=0.7)

# Adding labels and title
plt.xlabel('Alpha1')
plt.ylabel('RMSE')
plt.title('Boxplot of Validation RMSEs for various alphas')

# Show the plot
plt.show()

############################################################### PLOT 2
# Filter the DataFrame to exclude rows where alpha1 == 1e-20
filtered_val_plot = validation_results[validation_results['alpha1'] != 1e-20]

# Set the figure size
plt.figure(figsize=(12, 8))

# Create the boxplot with dots
sns.boxplot(x='alpha1', y='rmse', data=filtered_val_plot, palette="Set2")
sns.stripplot(x='alpha1', y='rmse', data=filtered_val_plot, color='black', jitter=True, alpha=0.7)

# Adding labels and title
plt.xlabel('Alpha1')
plt.ylabel('RMSE')
plt.title('Boxplot of Validation RMSEs for various alphas \n (Excluding alpha1 == 1e-20)')

# Show the plot
plt.show()

In [ ]:
#validation_results['alpha1'] = validation_results['alpha1'].astype(str)

In [ ]:
validation_avg = validation_results.groupby(['alpha1', 'lambda']).agg(
        rmse_mean=('rmse', 'mean'),
        rmse_sd=('rmse', 'std'),
        classification_error_mean=('classification_error', 'mean'),
        x2_minus_uv_norm_mean = ('x2_minus_uv_norm', 'mean'),
        v_minus_vhat_norm_mean = ('v_minus_vhat_norm', 'mean')
    ).reset_index()
validation_avg['rmse_lb'] = validation_avg['rmse_mean'] - validation_avg['rmse_sd']
validation_avg['rmse_ub'] = validation_avg['rmse_mean'] + validation_avg['rmse_sd']

##########################################
min_rmse_mean = validation_avg['rmse_mean'].min()
min_rmse_row = validation_avg[validation_avg['rmse_mean'] == min_rmse_mean]

lb = min_rmse_row['rmse_lb'].values[0]
ub = min_rmse_row['rmse_ub'].values[0]
#############################################

validation_avg['within_1sd_TF'] = (validation_avg['rmse_mean'] > lb) & (validation_avg['rmse_mean'] < ub)

validation_avg

In [ ]:
min_rmse_row

In [ ]:

# Filter the DataFrame to exclude rows where alpha1 == 1e-20
filtered_val_plot = validation_avg[validation_avg['alpha1'] != 1e-20]
filtered_val_plot['alpha1'] = filtered_val_plot['alpha1'].astype(str)

# Set the figure size
plt.figure(figsize=(12, 8))

# Plot the rmse_mean with error bars for rmse_lb and rmse_ub
plt.errorbar(
    x=filtered_val_plot['alpha1'], 
    y=filtered_val_plot['rmse_mean'], 
    yerr=[filtered_val_plot['rmse_mean'] - filtered_val_plot['rmse_lb'], 
          filtered_val_plot['rmse_ub'] - filtered_val_plot['rmse_mean']],
    fmt='o', capsize=6, capthick=3, color='blue', ecolor='black'
)

# Add horizontal lines at lb and ub
plt.axhline(y=lb, color='red', linestyle='--', label=f'Lower Bound: {lb:.4f}')
plt.axhline(y=ub, color='green', linestyle='--', label=f'Upper Bound: {ub:.4f}')

# Shade the area between the lb and ub
plt.fill_between(filtered_val_plot['alpha1'], lb, ub, color='gray', alpha=0.3)

for i, row in filtered_val_plot[filtered_val_plot['within_1sd_TF']].iterrows():
    plt.annotate(
        f'{row["rmse_mean"]:.2f}',  # Format rmse_mean to 2 decimal places
        (row['alpha1'], row['rmse_mean']), 
        textcoords="offset points", 
        xytext=(0, 20),  # Offset text above the data point
        ha='center', 
        fontsize=15, 
        color='red'
    )

min_alpha1 = min_rmse_row['alpha1'].values[0]
min_rmse_mean_val = min_rmse_row['rmse_mean'].values[0]

# Add a special annotation for the point with the lowest mean RMSE
plt.annotate(
    f'Alpha with the lowest mean RMSE: {min_alpha1} \nMean RMSE (+- 1sd)= {min_rmse_mean_val:.2f}({np.round(lb,2)}, {np.round(ub,2)})',  # Custom label text
    xy=(0.4, 0.6),  # Position in relative coordinates (0-1 range) or adjust as needed
    xycoords='axes fraction',  # Use relative coordinates
    fontsize=15, 
    color='black',
    bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white')
)

# Adding labels and title
plt.xlabel('Alpha')
plt.ylabel('RMSE')
plt.title('Using One Standard Deviation Rule to Select the Optimal Hyperparameter \n (Excluding alpha = 1e-20)', fontsize = 15)
plt.legend()

# Show the plot
plt.show()


In [ ]:
row

In [ ]:
# Create the first plot with rmse
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot rmse on the left y-axis
sns.pointplot(x='alpha1', y='rmse_mean', data=validation_results, marker='o', linestyles='-', ax=ax1, color='b')
ax1.set_xlabel('alpha1')
ax1.set_ylabel('rmse', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create a second y-axis for x2_minus_uv_norm
ax2 = ax1.twinx()
sns.pointplot(x='alpha1', y='x2_minus_uv_norm_mean', data=validation_results, marker='o', linestyles='-', ax=ax2, color='r')
ax2.set_ylabel('x2_minus_uv_norm', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add titles
plt.title('Validation- alpha vs RMSE and x2_minus_uv_norm')

# Show plot
plt.show()

In [ ]:
# Create the first plot with rmse
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot rmse on the left y-axis
sns.pointplot(x='alpha1', y='rmse_mean', data=validation_results, marker='o', linestyles='-', ax=ax1, color='b')
ax1.set_xlabel('alpha1')
ax1.set_ylabel('rmse', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create a second y-axis for x2_minus_uv_norm
ax2 = ax1.twinx()
sns.pointplot(x='alpha1', y='v_minus_vhat_norm_mean', data=validation_results, marker='o', linestyles='-', color='orange', ax=ax2)
ax2.set_ylabel('v_minus_vhat_norm', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

# Add titles
plt.title('Validation- alpha vs RMSE and v_minus_vhat_norm')

# Show plot
plt.show()

## 3. Training VS Validation results

In [ ]:
validation_results['alpha1'] = validation_results['alpha1'].astype(str)

# Select only the relevant columns from each dataset
train_rmse = train_results[['alpha1', 'rmse']]
validation_rmse = validation_results[['alpha1', 'rmse_mean']]


# Merge the datasets on the 'alpha1' column
rmse_tune = pd.merge(train_rmse, validation_rmse, on='alpha1')

rmse_tune.rename(columns={'rmse': 'rmse_train'}, inplace=True)
rmse_tune.rename(columns={'rmse_mean': 'rmse_val'}, inplace=True)

rmse_tune

In [ ]:
# Create the plot with rmse
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot rmse_train
sns.pointplot(x='alpha1', y='rmse_train', data=rmse_tune, marker='o', linestyles='-', ax=ax1, color='black', label='RMSE Train')

# Plot rmse_val
sns.pointplot(x='alpha1', y='rmse_val', data=rmse_tune, marker='o', linestyles='-', ax=ax1, color='blue', label='RMSE Validation')

# Set labels and title
ax1.set_xlabel('alpha1')
ax1.set_ylabel('rmse')
ax1.set_title('Training vs Validation RMSE over alpha')

# Add custom legend with the correct colors
ax1.legend(handles=[
    plt.Line2D([0], [0], color='black', marker='o', linestyle='-', label='RMSE Train'),
    plt.Line2D([0], [0], color='blue', marker='o', linestyle='-', label='RMSE Validation')
])

# Show plot
plt.show()

In [ ]:
# Select only the relevant columns from each dataset
train_x2_uv = train_results[['alpha1', 'x2_minus_uv_norm']]
validation_x2_uv = validation_results[['alpha1', 'x2_minus_uv_norm_mean']]


# Merge the datasets on the 'alpha1' column
x2_minus_uv_tune = pd.merge(train_x2_uv, validation_x2_uv, on='alpha1')

x2_minus_uv_tune.rename(columns={'x2_minus_uv_norm': 'x2_minus_uv_norm_train'}, inplace=True)
x2_minus_uv_tune.rename(columns={'x2_minus_uv_norm_mean': 'x2_minus_uv_norm_val'}, inplace=True)

x2_minus_uv_tune

In [ ]:
# Create the plot with rmse
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot rmse_train
sns.pointplot(x='alpha1', y='x2_minus_uv_norm_train', data=x2_minus_uv_tune, marker='o', linestyles='-', ax=ax1, color='black', label='x2_minus_uv_tune Train')

# Plot rmse_val
sns.pointplot(x='alpha1', y='x2_minus_uv_norm_val', data=x2_minus_uv_tune, marker='o', linestyles='-', ax=ax1, color='r', label='x2_minus_uv_tune Validation')

# Set labels and title
ax1.set_xlabel('alpha1')
ax1.set_ylabel('x2_minus_uv_tune')
ax1.set_title('Training vs Validation x2_minus_uv_tune over alpha')

# Add custom legend with the correct colors
ax1.legend(handles=[
    plt.Line2D([0], [0], color='black', marker='o', linestyle='-', label='x2_minus_uv_tune Train'),
    plt.Line2D([0], [0], color='r', marker='o', linestyle='-', label='x2_minus_uv_tune Validation')
])

# Show plot
plt.show()

In [ ]:
# Select only the relevant columns from each dataset
train_v = train_results[['alpha1', 'v_minus_vhat_norm']]
validation_v= validation_results[['alpha1', 'v_minus_vhat_norm_mean']]


# Merge the datasets on the 'alpha1' column
v_minus_vhat_tune = pd.merge(train_v, validation_v, on='alpha1')

v_minus_vhat_tune.rename(columns={'v_minus_vhat_norm': 'v_minus_vhat_norm_train'}, inplace=True)
v_minus_vhat_tune.rename(columns={'v_minus_vhat_norm_mean': 'v_minus_vhat_norm_val'}, inplace=True)

v_minus_vhat_tune

In [ ]:
# Create the plot with rmse
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot rmse_train
sns.pointplot(x='alpha1', y='v_minus_vhat_norm_train', data=v_minus_vhat_tune, marker='o', linestyles='-', ax=ax1, color='black', label='v_minus_vhat_norm Train')

# Plot rmse_val
sns.pointplot(x='alpha1', y='v_minus_vhat_norm_val', data=v_minus_vhat_tune, marker='o', linestyles='-', ax=ax1, color='orange', label='v_minus_vhat_norm Validation')

# Set labels and title
ax1.set_xlabel('alpha1')
ax1.set_ylabel('v_minus_vhat_norm_train')
ax1.set_title('Training vs Validation v_minus_vhat_norm_train over alpha')

# Add custom legend with the correct colors
ax1.legend(handles=[
    plt.Line2D([0], [0], color='black', marker='o', linestyle='-', label='v_minus_vhat_norm Train'),
    plt.Line2D([0], [0], color='orange', marker='o', linestyle='-', label='v_minus_vhat_norm Validation')
])

# Show plot
plt.show()

In [ ]:
optimal_hyperparam_report

In [ ]:
vsignal = 10
noise = 0.2

main_topic_signal = 10
topic_weights = [2,2, 1, 1, 4]  # Adjust weights as desired

# RCT
gamma1 = gamma2 = 0 

psi1= 10
psi2= 5
beta0 = 0
beta1 = 0
beta2 = 0
beta3 = -30
beta4 = 0
beta5 = 0
rho1 = 0 #Interaction effect between words in topic 3 and topic 4: X2_41*X2_42*X2_61* X2_62 
rho2 = 0 #Interaction effect between words in topic 1 and 2
rho3 = 0 #TBD

n, m1, m2, R = 2000, 10, 100, 5

eta= noise #controls noise level 
epsilon = noise #controls noise level 

v_signal_val = vsignal

seed_train = 2
seed_validation1 = 4
seed_validation2 =11
seed_test = 12

data_train, Vtrue_train, NA, rmse_min_train = generate_data_dlearning(seed=seed_train, n = n, r=R, m1=m1, m2 = m2, zeta1= 1, zeta2=2,\
                                main_topic_signal=main_topic_signal,topic_weights \
                                = topic_weights,eta =eta, epsilon=epsilon, gamma1=gamma1, gamma2=gamma2, \
                                beta0=beta0, psi1 = psi1, psi2=psi2,beta1=beta1, beta2=beta2,  beta3=beta3, beta4 = beta4, beta5 = beta5, rho1 =rho1, rho2=rho2, rho3= rho3, v_signal_val=v_signal_val, scenario = "A")

data_validation1, Vtrue_validation1, NA, rmse_min_val1 = generate_data_dlearning(seed=seed_validation1, n = n, r=R, m1=m1, m2 = m2, zeta1= 1, zeta2=2,\
                                main_topic_signal=main_topic_signal,topic_weights \
                                = topic_weights,eta =eta, epsilon=epsilon, gamma1=gamma1, gamma2=gamma2, \
                               beta0=beta0, psi1 = psi1, psi2=psi2,beta1=beta1, beta2=beta2,  beta3=beta3, beta4 = beta4, beta5 = beta5, rho1 =rho1, rho2=rho2, rho3= rho3, v_signal_val=v_signal_val, scenario = "A")

data_validation2, Vtrue_validation2, NA, rmse_min_val2 = generate_data_dlearning(seed=seed_validation2, n = n, r=R, m1=m1, m2 = m2, zeta1= 1, zeta2=2,\
                                main_topic_signal=main_topic_signal,topic_weights \
                                = topic_weights,eta =eta, epsilon=epsilon, gamma1=gamma1, gamma2=gamma2, \
                               beta0=beta0, psi1 = psi1, psi2=psi2,beta1=beta1, beta2=beta2,  beta3=beta3, beta4 = beta4, beta5 = beta5, rho1 =rho1, rho2=rho2, rho3= rho3, v_signal_val=v_signal_val, scenario = "A")


data_test, Vtrue_test, NA, rmse_min_test= generate_data_dlearning(seed=seed_test, n = n, r=R, m1=m1, m2 = m2, zeta1= 1, zeta2=2,\
                            main_topic_signal=main_topic_signal,topic_weights \
                            = topic_weights,eta =eta, epsilon=epsilon, gamma1=gamma1, gamma2=gamma2, \
                            beta0=beta0, psi1 = psi1, psi2=psi2,beta1=beta1, beta2=beta2,  beta3=beta3, beta4 = beta4, beta5 = beta5, rho1 =rho1, rho2=rho2, rho3= rho3, v_signal_val=v_signal_val, scenario = "A")


In [ ]:
output_optimal = output[7][0] # since there is only one output

V_snmf = output_optimal[3]

V_snmf_r = reorder_V(Vtrue_train, V_snmf, m2, True)
V_snmf_R = reorder_V(Vtrue_train, V_snmf, m2, False)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

v_error = train_results['v_minus_vhat_norm']
rmse = train_results['rmse']

num_alphas = len(alpha_values)
num_cols = 6  # Adjust the number of columns as needed
num_rows = (num_alphas + num_cols - 1) // num_cols  # Calculate the number of rows needed

fig, axes = plt.subplots(num_rows, num_cols, figsize=(30, 15))  # Adjust the overall figure size
axes = axes.flatten()  # Flatten the 2D array of axes for easy indexing

for alpha_index in range(num_alphas):
    alpha = alpha_values[alpha_index]
    output_optimal = output[alpha_index][0]  # since there is only one output
    v_error_i = np.round(v_error[alpha_index],2)
    rmse_i = np.round(rmse[alpha_index],2)
    V_snmf = output_optimal[3]
    
    V_snmf_r = reorder_V(Vtrue_train, V_snmf, m2, True)
    
    norms = np.linalg.norm(V_snmf_r, axis=1, keepdims=True)
    V_snmf_r_norm = V_snmf_r / norms
    V_snmf_df = pd.DataFrame(V_snmf_r_norm.T)
    num_rows_df = len(V_snmf_df)
    V_snmf_df['word'] = ['word_' + str(i+1) for i in range(num_rows_df)]
    
    new_column_names = {
        0: 'topic_1',
        1: 'topic_2',
        2: 'topic_3',
        3: 'topic_4',
        4: 'topic_5',
    }

    # Rename all columns
    V_snmf_df.rename(columns=new_column_names, inplace=True)
    V_snmf_df.set_index('word', inplace=True)

    sns.heatmap(V_snmf_df, annot=False, cmap='Blues', fmt='.2f', cbar=False, ax=axes[alpha_index])
    axes[alpha_index].set_title(f'Alpha = {alpha}, \n V error = {v_error_i}, \n rmse = {rmse_i}')

# Remove any unused subplots
for i in range(num_alphas, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.savefig(f'/ihome/ltang/ziz60/nmf_causal_eval/21_simple_scenario_interaction/output/figure/V_hat_heatmap_alpha_tuning.png')

plt.show()


---------------------------------------

In [ ]:
def run_2s_nmf(data_train, data_test, m1, m2, R, r):
    # RCT
    gamma1 = gamma2 = 0 
    
    #######################################################
    ###################  Training #########################
    #######################################################
    
    delta_x = data_train["delta_x"]
    delta_x = vec2col(delta_x)
    Y = data_train[['y']].values  # array
    A = data_train[['A']].values  # array
    X = data_train.iloc[:, 0:m1+m2]
    X1 = data_train.iloc[:, 0:m1]  # EHR data
    X2 = data_train.iloc[:, m1:m1+m2]  # document word
    U = data_train.iloc[:, (m1+m2+1):(m1+m2 + R + 1)]  # no intercept

    # format dataframe to np array
    X = X.to_numpy()
    X1 = X1.to_numpy()
    X2 = X2.to_numpy()
    U = U.to_numpy()
    U_true = U.copy()

    # STEP 1. Estimate U and V using NMF, with NNDSVD initialization
    model = NMF(n_components=r, init='nndsvd', random_state=10, solver='cd')
    U_out = model.fit_transform(X2)  # n times r
    V_out = model.components_

    X = np.concatenate((X1, U_out), axis=1)  # not reordered

    # Estimate treatment propensity p_hat
    if gamma1 == gamma2 == 0:
        data_train["pi_hat"] = 0.5
    else:
        # estimate p_hat using logistic regression, regression data on X (EHR and initial U)
        model = LogisticRegression(solver='liblinear', random_state=1)
        model.fit(X, A)

        pi_hat = model.predict_proba(X)[:, 1]  # p_hat = estimated probability A=1
        pi_hat = vec2col(pi_hat)
        data_train["pi_hat"] = pi_hat

    W_hat = np.where(data_train['A'] == 1, 1 / data_train['pi_hat'], 1 / (1 - data_train['pi_hat']))
    data_train['W_hat'] = W_hat
    W_hat = vec2col(W_hat)
    W = np.eye(len(W_hat))  # make W a diagonal matrix
    np.fill_diagonal(W, W_hat)

    ########## TRANSFORMATION
    Y_tilde = 2 * data_train['y'] * data_train['A'] * np.sqrt(data_train["W_hat"])  # transform Y
    Y_tilde = vec2col(Y_tilde)

    scaler = StandardScaler()

    X_tilde = np.sqrt(W).dot(X)
    X_tilde = scaler.fit_transform(X_tilde)

    # Step 2. Regress X1, U on Y to estimate coefficients
    model = LinearRegression().fit(X_tilde, Y_tilde)

    beta0_hat = model.intercept_ / 2  # intercept
    beta_hat = model.coef_ / 2  # beta coefficients

    beta_out = np.concatenate((beta0_hat, beta_hat[0]))

    twostage_intercept = np.round(beta_out[0], 2) / 2
    twostage_beta = np.round(beta_out[11:] / 2, 2).tolist()

    # generate_cor_df(Vtrue_train, V_out, R, r)  # not reordered

    ## Training eval
    delta_x_hat_train = beta_out[0] + scaler.fit_transform(X1).dot(beta_out[1:m1+1]) + scaler.fit_transform(U_out).dot(beta_out[m1+1:])
    d_x_hat_train = np.where(delta_x_hat_train > 0, 1, -1)
    d_x_hat_train = d_x_hat_train.flatten()
    d_x = data_train["d_x"]
    NA, NA, NA, NA, NA, NA, accuracy_train = eval_ITR(d_x, d_x_hat_train)

    # print("Training Classification Error:", 1-accuracy_train)
    # print(pd.crosstab(d_x, d_x_hat_train))
    rmse_train = np.sqrt(np.linalg.norm(2 * Y * A - delta_x_hat_train) ** 2)
    # print("Training RMSE:", rmse_train)

    #######################################################
    ###################  Test #########################
    #######################################################
    delta_x = data_test["delta_x"]
    delta_x = vec2col(delta_x)
    Y = data_test[['y']].values  # array
    A = data_test[['A']].values  # array
    X1 = data_test.iloc[:, 0:m1]  # EHR data
    X2 = data_test.iloc[:, m1:m1+m2]  # document word

    # format dataframe to np array
    X1 = X1.to_numpy()
    X2 = X2.to_numpy()

    # U test
    model = NMF(n_components=r, init='random', random_state=1)
    Uinit = model.fit_transform(X2)  # n times r

    # Not reordered U
    step_size = 1  # updating a
    step_size_a = 1
    step_size_b = 0.1
    step_size_sigma = 0.01
    U_out, convergence_step = gradient_descent_test(X2, X, W, Y, Y_tilde, Uinit, V_out, Vtrue_test, r, beta_out, 0, 0, 1e-20, 400, step_size_a, step_size_b, step_size_sigma, False)

    U_out_r = reorder_U(Vtrue_train, V_out, U_out, m2, True)
    U_out_R = reorder_U(Vtrue_train, V_out, U_out, m2, False)

    U_2stage_test_r = U_out_R

    ## reordering estimated beta and U 
    V_2stage_r = reorder_V(Vtrue_train, V_out, m2, True)
    V_2stage_R = reorder_V(Vtrue_train, V_out, m2, False)

    beta_2stage_r = reorder_beta(Vtrue_train, V_out, beta_out, m1, m2, True)
    beta_2stage_R = reorder_beta(Vtrue_train, V_out, beta_out, m1, m2,  False)

    delta_x_hat = beta_2stage_r[0] + scaler.fit_transform(X1).dot(beta_2stage_r[1:m1+1]) + scaler.fit_transform(U_out_r).dot(beta_2stage_r[m1+1:])

    d_x_hat = np.where(delta_x_hat > 0, 1, -1)
    d_x_hat = d_x_hat.flatten()

    d_x = data_test["d_x"]
    NA, NA, NA, NA, NA, NA, accuracy_test = eval_ITR(d_x, d_x_hat)
    rmse_test = np.sqrt(np.linalg.norm(2 * Y * A - delta_x_hat) ** 2)

    return accuracy_train, rmse_train, accuracy_test, rmse_test, V_2stage_r, beta_2stage_r


In [ ]:
accuracy_train_2s, rmse_train_2s, accuracy_test_2s, rmse_test_2s, V_2stage_r, beta_2stage_r = run_2s_nmf(data_train, data_test, m1, m2, R, r = 5)

print("Training: accuracy = ", accuracy_train_2s)
print("Training: rmse = ", rmse_train_2s)
print("Testin: accuracy = ", accuracy_test_2s)
print("Testin: rmse = ", rmse_test_2s)

In [ ]:
# V plot -- top words
# takes three V matrics: 1. Vtrue_train, 2.V_2stage_r, 3.V_snmf_r
Vtrue_df = Vtrue_train.T
Vtrue_df.reset_index(inplace=True)
# Rename the column 'index' to whatever you prefer, e.g., 'row_name'
Vtrue_df.rename(columns={'index': 'word'}, inplace=True)
# Create a dictionary with the new column names
new_column_names = {
    1: 'topic_1',
    2: 'topic_2',
    3: 'topic_3',
    4: 'topic_4',
    5: 'topic_5',
}
# Rename all columns
Vtrue_df.rename(columns=new_column_names, inplace=True)
###################################################################
norms = np.linalg.norm(V_2stage_r, axis=1, keepdims=True)
V_2stage_r_norm = V_2stage_r/ norms
V_2s_df = pd.DataFrame(V_2stage_r_norm.T)
    
num_rows = len(V_2s_df)
V_2s_df[['word']] = ['word_' + str(i+1) for i in range(num_rows)]

# Create a dictionary with the new column names
new_column_names = {
    0: 'topic_1',
    1: 'topic_2',
    2: 'topic_3',
    3: 'topic_4',
    4: 'topic_5',
}

# Rename all columns
V_2s_df.rename(columns=new_column_names, inplace=True)
###################################################################
norms = np.linalg.norm(V_snmf_r, axis=1, keepdims=True)
V_snmf_r_norm = V_snmf_r/ norms
V_snmf_df = pd.DataFrame(V_snmf_r_norm.T)
num_rows = len(V_snmf_df)
V_snmf_df[['word']] = ['word_' + str(i+1) for i in range(num_rows)]
new_column_names = {
    0: 'topic_1',
    1: 'topic_2',
    2: 'topic_3',
    3: 'topic_4',
    4: 'topic_5',
}

# Rename all columns
V_snmf_df.rename(columns=new_column_names, inplace=True)

In [ ]:
Vtrue_df

In [ ]:
# Set 'word' column as the index to use it as row labels
V_2s_df.set_index('word', inplace=True)
V_snmf_df.set_index('word', inplace=True)
Vtrue_df.set_index('word', inplace=True)

In [ ]:
V_snmf_df

In [ ]:
plt.figure(figsize=(4, 5))  # Adjust the width and height as needed

sns.heatmap(V_snmf_df, annot=False, cmap='Blues', fmt='.2f', cbar=False)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['text.usetex'] = False
# Create the subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 12))

# Plot each heatmap
# 'crest', 'coolwarm', cmap=sns.cubehelix_palette(as_cmap=True, start=0), 
sns.heatmap(Vtrue_df, annot=False, cmap='Blues', fmt='.2f', ax=axs[0], cbar=False)
sns.heatmap(V_snmf_df, annot=False, cmap='Blues', fmt='.2f', ax=axs[1], cbar=False)
sns.heatmap(V_2s_df, annot=False, cmap='Blues', fmt='.2f', ax=axs[2], cbar=True)

# Set titles
axs[0].set_title('True $V^T$')
axs[1].set_title(r'$\hat{V}^T$ - SNMF')
axs[2].set_title(r'$\hat{V}^T$ - 2-Stage NMF')

# Set common labels
for ax in axs:
    ax.set_xlabel('Topics')
    ax.set_ylabel('Words')

# Remove y-axis labels from all but the first plot
for ax in axs[1:]:
    ax.set_ylabel('')

plt.tight_layout()
#plt.savefig(f'/ihome/ltang/ziz60/nmf_causal_eval/21_simple_scenario_interaction/output/figure/heatmap_v.png')

plt.show()